# Build a function template for Hamiltonian simulation

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=1.2.4
qiskit-ibm-runtime~=0.32.0
qiskit-serverless~=0.18.0
qiskit-ibm-catalog~=0.2.0
qiskit-addon-aqc-tensor[quimb-jax]~=0.1.0
mergedeep==1.3.4
```
</details>

By now you've seen a basic example of how to get started writing, uploading, and running a program with Qiskit Serverless. The workloads you are building for your own use cases are likely not so simple. For example, you might need to carefully consider what domain-level inputs and outputs are relevant for your application, how to make sure your program can be reusable across a range of those inputs, and what kind of information you need returned to you during the execution of your job so that you can better evaluate the progress of your workload. You may even want to incorporate a dry run mode that lets you test the impact of a set of particular inputs before sending the resulting circuits to hardware.


A function template is an example of a realistic workload using a specific application domain to contextualize these aspects. It is meant to be a starting point for you to take and modify for your own needs so that you don't have to start from scratch.


Here, we will see a function template for addressing Hamiltonian simulation problems.

## Write the function template

We will write a function template for Hamiltonian simulation using the AQC-Tensor Qiskit addon to map the problem description to a reduced-depth circuit for execution on hardware.

We will save the following code to `./source_files/function_template_hamsim.py`. This file is the function template we'll upload to and run remotely with Qiskit Serverless.

In [1]:
# This cell is hidden from users, it just creates a new folder
from pathlib import Path

Path("./source_files").mkdir(exist_ok=True)

### Get the domain-level inputs

We start by getting the inputs for our template. In this case, we have domain-specific inputs relevant for Hamiltonian simulation (such as the Hamiltonian and observable) and capability-specific options (such as how much we want to compress the initial layers of our Trotter circuit using AQC-Tensor or advanced options for fine-tuning error suppression and mitigation beyond the defaults we have baked into for this example).

In [2]:
%%writefile ./source_files/function_template_hamsim.py

from qiskit import QuantumCircuit
from qiskit_serverless import get_arguments, save_result


# Extract parameters from arguments
#
# We do this at the top of the program so we can fail early if any required arguments are missing or invalid.

arguments = get_arguments()

dry_run = arguments.get("dry_run", False)
backend_name = arguments["backend_name"]

aqc_evolution_time = arguments["aqc_evolution_time"]
aqc_ansatz_num_trotter_steps = arguments["aqc_ansatz_num_trotter_steps"]
aqc_target_num_trotter_steps = arguments["aqc_target_num_trotter_steps"]

remainder_evolution_time = arguments["remainder_evolution_time"]
remainder_num_trotter_steps = arguments["remainder_num_trotter_steps"]

# Stop if this fidelity is achieved
aqc_stopping_fidelity = arguments.get("aqc_stopping_fidelity", 1.0)
# Stop after this number of iterations, even if stopping fidelity is not achieved
aqc_max_iterations = arguments.get("aqc_max_iterations", 500)

hamiltonian = arguments["hamiltonian"]
observable = arguments["observable"]
initial_state = arguments.get("initial_state", QuantumCircuit(hamiltonian.num_qubits))

Overwriting ./source_files/function_template_hamsim.py


In [4]:
%%writefile --append ./source_files/function_template_hamsim.py

import numpy as np
import json
from mergedeep import merge


# Configure `EstimatorOptions`, to control the parameters of our hardware experiment
#
# Set default options
estimator_default_options = {
    "resilience": {
        "measure_mitigation": True,
        "zne_mitigation": True,
        "zne": {
            "amplifier": "gate_folding",
            "noise_factors": [1, 2, 3],
            "extrapolated_noise_factors": list(np.linspace(0, 3, 31)),
            "extrapolator": ["exponential", "linear", "fallback"],
        },
        "measure_noise_learning": {
            "num_randomizations": 512,
            "shots_per_randomization": 512,
        },
    },
    "twirling": {
        "enable_gates": True,
        "enable_measure": True,
        "num_randomizations": 300,
        "shots_per_randomization": 100,
        "strategy": "active",
    },
}
# Merge with user-provided options
estimator_options = merge(
    arguments.get("estimator_options", {}), estimator_default_options
)


Appending to ./source_files/function_template_hamsim.py


When the function template is running, it is helpful to return information in the logs, using print statements, so that you can better evaluate the progress of your workload. Here is a simple example of printing the estimator_options so that there is a record of the actual Estimator options used. There are many more example like this throughout the program to report back progress during execution. Including the value of the objective function during the iterative component of AQC-Tensor, or the two-qubit depth of the final ISA circuit intended for execution on hardware.

In [ ]:
print("estimator_options =", json.dumps(estimator_options, indent=4))

### Validate the inputs

An important aspect of ensuring reusability of the template across a range of inputs is to validate the inputs. Here is an example of how we check that the stopping fidelity during AQC-Tensor has been specified appropriately and, if not, return an informative error message for how to fix the error.

In [ ]:
%%writefile --append ./source_files/function_template_hamsim.py

# Perform parameter validation

if not 0.0 < aqc_stopping_fidelity <= 1.0:
    raise ValueError(
        f"Invalid stopping fidelity: {aqc_stopping_fidelity} - must be positive float no greater than 1"
    )

This `print()` call is also the first example of logging, so that there is a record of the actual Estimator options used.  There are many more throughout the program below to report progress throughout the job.

### Perform the Hamiltonian simulation workflow using AQC-Tensor

First, we prepare a dictionary to hold all output of the function template. Keys will be added to this dict throughout the workflow, and it will be returned at the conclusion of the program.

In [5]:
%%writefile --append ./source_files/function_template_hamsim.py

output = {}

Appending to ./source_files/function_template_hamsim.py


#### Step 1: Map

The AQC-Tensor optimization happens in step 1 of a Qiskit pattern.  First, a target state is constructed, here from a target circuit that evolves the same hamiltonian for the same time period as the AQC portion.  Then, an ansatz is generated from an equivalent circuit but with fewer Trotter steps.  In the main portion of the AQC algorithm, that ansatz is brought closer to the target state in an iterative process.  Then, the result is combined with the remainder of the Trotter steps needed to reach the desired evolution time.

Note the additional examples of logging incorporated below.

In [6]:
%%writefile --append ./source_files/function_template_hamsim.py

import datetime
import quimb.tensor
from scipy.optimize import OptimizeResult, minimize
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import generate_time_evolution_circuit
from qiskit_addon_aqc_tensor.ansatz_generation import (
    generate_ansatz_from_circuit,
    AnsatzBlock,
)
from qiskit_addon_aqc_tensor.simulation import (
    tensornetwork_from_circuit,
    compute_overlap,
)
from qiskit_addon_aqc_tensor.simulation.quimb import QuimbSimulator
from qiskit_addon_aqc_tensor.objective import OneMinusFidelity

print("Hamiltonian:", hamiltonian)
print("Observable:", observable)
simulator_settings = QuimbSimulator(quimb.tensor.CircuitMPS, autodiff_backend="jax")

# Construct the AQC target circuit
aqc_target_circuit = initial_state.copy()
if aqc_evolution_time:
    aqc_target_circuit.compose(
        generate_time_evolution_circuit(
            hamiltonian,
            synthesis=SuzukiTrotter(reps=aqc_target_num_trotter_steps),
            time=aqc_evolution_time,
        ),
        inplace=True,
    )

# Construct matrix-product state representation of the AQC target state
aqc_target_mps = tensornetwork_from_circuit(aqc_target_circuit, simulator_settings)
print("Target MPS maximum bond dimension:", aqc_target_mps.psi.max_bond())
output["target_bond_dimension"] = aqc_target_mps.psi.max_bond()

# Generate an ansatz and initial parameters from a Trotter circuit with fewer steps
aqc_good_circuit = initial_state.copy()
if aqc_evolution_time:
    aqc_good_circuit.compose(
        generate_time_evolution_circuit(
            hamiltonian,
            synthesis=SuzukiTrotter(reps=aqc_ansatz_num_trotter_steps),
            time=aqc_evolution_time,
        ),
        inplace=True,
    )
aqc_ansatz, aqc_initial_parameters = generate_ansatz_from_circuit(aqc_good_circuit)
print("Number of AQC parameters:", len(aqc_initial_parameters))
output["num_aqc_parameters"] = len(aqc_initial_parameters)

# Calculate fidelity of ansatz circuit vs. the target state, before optimization
good_mps = tensornetwork_from_circuit(aqc_good_circuit, simulator_settings)
starting_fidelity = abs(compute_overlap(good_mps, aqc_target_mps)) ** 2
print("Starting fidelity of AQC portion:", starting_fidelity)
output["aqc_starting_fidelity"] = starting_fidelity

# Optimize the parameters of the ansatz using MPS calculations
def callback(intermediate_result: OptimizeResult):
    fidelity = 1 - intermediate_result.fun
    print(f"{datetime.datetime.now()} Intermediate result: Fidelity {fidelity:.8f}")
    if intermediate_result.fun < stopping_point:
        raise StopIteration


objective = OneMinusFidelity(aqc_target_mps, aqc_ansatz, simulator_settings)
stopping_point = 1.0 - aqc_stopping_fidelity

result = minimize(
    objective,
    aqc_initial_parameters,
    method="L-BFGS-B",
    jac=True,
    options={"maxiter": aqc_max_iterations},
    callback=callback,
)
if result.status not in (
    0,
    1,
    99,
):  # 0 => success; 1 => max iterations reached; 99 => early termination via StopIteration
    raise RuntimeError(
        f"Optimization failed: {result.message} (status={result.status})"
    )
print(f"Done after {result.nit} iterations.")
output["num_iterations"] = result.nit
aqc_final_parameters = result.x
output["aqc_final_parameters"] = list(aqc_final_parameters)

# Construct optimized circuit for initial portion of time evolution
aqc_final_circuit = aqc_ansatz.assign_parameters(aqc_final_parameters)

# Calculate fidelity after optimization
aqc_final_mps = tensornetwork_from_circuit(aqc_final_circuit, simulator_settings)
aqc_fidelity = abs(compute_overlap(aqc_final_mps, aqc_target_mps)) ** 2
print("Fidelity of AQC portion:", aqc_fidelity)
output["aqc_fidelity"] = aqc_fidelity

# Construct final circuit, with remainder of time evolution
final_circuit = aqc_final_circuit.copy()
if remainder_evolution_time:
    remainder_circuit = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(reps=remainder_num_trotter_steps),
        time=remainder_evolution_time,
    )
    final_circuit.compose(remainder_circuit, inplace=True)

Appending to ./source_files/function_template_hamsim.py


#### Step 2: Optimize

After the AQC portion of the workflow, the `final_circuit` is transpiled to hardware as usual.

In [7]:
%%writefile --append ./source_files/function_template_hamsim.py

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.backend(backend_name)

# Transpile pubs (circuits and observables) to match ISA
pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pass_manager.run(final_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

isa_2qubit_depth = isa_circuit.depth(lambda x: x.operation.num_qubits == 2)
print("ISA circuit two-qubit depth:", isa_2qubit_depth)
output["twoqubit_depth"] = isa_2qubit_depth

Appending to ./source_files/function_template_hamsim.py


#### Exit early if using dry run mode

If dry run mode has been selected then the program is stopped before executing on hardware. This can be useful if, for example, you want first to inspect the two-qubit depth of the ISA circuit before deciding to execute on hardware.

In [8]:
%%writefile --append ./source_files/function_template_hamsim.py

# Exit now if dry run; don't execute on hardware
if dry_run:
    import sys

    print("Aborting before hardware execution since `dry_run` is True.")
    save_result(output)
    sys.exit(0)

Appending to ./source_files/function_template_hamsim.py


#### Step 3: Execute on hardware

In [9]:
%%writefile --append ./source_files/function_template_hamsim.py

# ## Step 3: Execute quantum experiments on backend
from qiskit_ibm_runtime import EstimatorV2 as Estimator


estimator = Estimator(backend, options=estimator_options)

# Submit the underlying Estimator job. Note that this is not the
# actual function job.
job = estimator.run([(isa_circuit, isa_observable)])
print("Job ID:", job.job_id())
output["job_id"] = job.job_id()

# Wait until job is complete
hw_results = job.result()
hw_results_dicts = [pub_result.data.__dict__ for pub_result in hw_results]

# Save hardware results to serverless output dict
output["hw_results"] = hw_results_dicts

# Re-organize expectation values
hw_expvals = [pub_result_data["evs"].tolist() for pub_result_data in hw_results_dicts]

# Save expectation values to serverless
print("Hardware expectation values", hw_expvals)
output["hw_expvals"] = hw_expvals[0]

Appending to ./source_files/function_template_hamsim.py


#### Save the output

This function template returns the relevant domain-level output for this Hamiltonian simulation workflow (expectaton values) in addition to important metadata generated along the way.

In [10]:
%%writefile --append ./source_files/function_template_hamsim.py

save_result(output)

Appending to ./source_files/function_template_hamsim.py


This workflow grew out of a Jupyter notebook.  Link to it. FIXME

## Deploy to IBM Quantum Platform

The previous section created a program to be run remotely. The code cells in this section upload that program to Qiskit Serverless.

Use `qiskit-ibm-catalog` to authenticate to `QiskitServerless` with your API token, which you can find in your [IBM Quantum account](https://quantum.ibm.com/account), and upload the program.

You can use `save_account()` to save your credentials (See the "Authenticate to the service" step in the [Set up to use IBM Quantum Platform](./setup-channel#set-up-to-use-ibm-quantum-platform) section). Note that this writes your credentials to the same file as [`QiskitRuntimeService.save_account()`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#save_account).

In [11]:
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction

# Authenticate to the remote cluster and submit the pattern for remote execution
serverless = QiskitServerless()

This program has custom `pip` dependencies, so we add them to a `dependencies` array when constructing the `QiskitFunction` instance:

In [29]:
template = QiskitFunction(
    title="function_template_hamsim",
    entrypoint="hfunction_template_hamsim.py",
    working_dir="./source_files/",
    dependencies=[
        "qiskit-addon-utils==0.1.0",
        "qiskit-addon-aqc-tensor[quimb-jax]==0.1.0",
        "mergedeep",
    ],
)

In [30]:
serverless.upload(template)

QiskitFunction(function_template_hamsim)

To check if the program successfully uploaded, use `serverless.list()`:

In [31]:
serverless.list()

[QiskitFunction(function_template_hamsim)]

## Run the function template remotely

Now that we have uploaded the function template, we can run it remotely with Qiskit Serverless. First, we load the template by name:

In [32]:
template = serverless.load("function_template_hamsim")

Then, we run the template with the domain-level inputs for Hamiltonian simulation. In this example, we specify a simple 4-qubit hamiltonian along with an observable.

In [33]:
from qiskit.quantum_info import SparsePauliOp

hamiltonian = SparsePauliOp.from_sparse_list(
    [("XX", (i, i + 1), 1.0) for i in range(3)], num_qubits=4
) + SparsePauliOp.from_sparse_list(
    [("YY", (i, i + 1), 1.0) for i in range(3)], num_qubits=4
)
observable = SparsePauliOp.from_sparse_list([("ZZ", (1, 2), 1.0)], num_qubits=4)

In [34]:
job = template.run(
    dry_run=True,
    hamiltonian=hamiltonian,
    observable=observable,
    backend_name="ibm_fez",
    estimator_options={},
    aqc_evolution_time=0.2,
    aqc_ansatz_num_trotter_steps=1,
    aqc_target_num_trotter_steps=32,
    remainder_evolution_time=0.2,
    remainder_num_trotter_steps=4,
    aqc_max_iterations=300,
)
print(job.job_id)

bd22fb47-ee0d-499f-8e85-2b55b526e23b


We can check the status of our job:

In [ ]:
job.status()

Once the job is running, we can fetch logs created from the `print()` outputs. These can provide actionable information about the progress of our Hamiltonian simulation workflow. For example, the value of the objective function during the iterative component of AQC, or the two-qubit depth of the final ISA circuit intended for execution on hardware.

In [ ]:
job.logs()

Block until a result is available. And once the job is done, we can retrieve the results. These include the domain-level output of Hamiltonian simulation (expectation value) and useful metadata.

In [ ]:
job.result()

## Next steps

<Admonition type="info" title="Recommendations">

- See an [example of this function template](https://qiskit.github.io/qiskit-addon-aqc-tensor/) in action on a large-scale Hamiltonian simulation problem. 
{/* - For a deeper dive into the AQC-Tensor Qiskit addon, check out the tutorial on the Learning Platform (add-link-when-live). */}

</Admonition>

In [ ]:
# This cell is hidden from users, it just deletes the working folder we created
import shutil

shutil.rmtree("./source_files/")